In [1]:
!pip install -U onnx-tf

In [2]:
!pip install torch

In [3]:
!pip install tensorflow

In [4]:
!pip install tensorflow_probability

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import onnx
import tensorflow as tf
import onnx_tf

2024-03-02 10:05:37.725039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/user/anaconda3/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.block1 = nn.Sequential(
            # 3x28x28
            nn.Conv2d(in_channels=3,
                      out_channels=16,
                      kernel_size=5,
                      stride=1,
                      padding=2),
            # 16x28x28
            nn.MaxPool2d(kernel_size=2),
            # 16x14x14
            nn.LeakyReLU()
        )
        # 16x14x14
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                      out_channels=32,
                      kernel_size=5,
                      stride=1,
                      padding=2),
            # 32x14x14
            nn.MaxPool2d(kernel_size=2),
            # 32x7x7
            nn.LeakyReLU()
        )
        # linearly
        self.block3 = nn.Sequential(
            nn.Linear(32 * 7 * 7, 100),
            nn.LeakyReLU(),
            nn.Linear(100, 37)
        )
        # 1x36

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        # flatten the dataset
        # ipdb; ipdb.set_trace()
        out = out.view(-1, 32 * 7 * 7)
        out = self.block3(out)

        return out

In [7]:
pytorch_model = CNN()
pytorch_model.load_state_dict(torch.load("./model.pth"))

<All keys matched successfully>

In [8]:
pytorch_model.eval()

CNN(
  (block1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (block3): Sequential(
    (0): Linear(in_features=1568, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=37, bias=True)
  )
)

In [9]:
# Export the PyTorch model to ONNX format
input_shape = (5, 3, 28, 28)
dummy_input = torch.randn(input_shape)
onnx_model_path = 'model.onnx'
torch.onnx.export(pytorch_model, dummy_input, onnx_model_path, verbose=False)

In [10]:
# Load the ONNX model
onnx_model = onnx.load(onnx_model_path)

In [11]:
# Convert the ONNX model to TensorFlow format
# Error cause: KeyError: 'input.1'
# tf_model_path = 'model.pb'
# tf_rep = onnx_tf.backend.prepare(onnx_model)
# tf_rep.export_graph(tf_model_path)

In [12]:
# Error
print("Model Inputs: ", [inp.name for inp in onnx_model.graph.input])

Model Inputs:  ['input.1']


In [13]:
# Change input.1 ke input_1
import onnx
from onnx import helper

onnx_model = onnx.load(onnx_model_path)

# Define a mapping from old names to new names
name_map = {"input.1": "input_1"}

# Initialize a list to hold the new inputs
new_inputs = []

# Iterate over the inputs and change their names if needed
for inp in onnx_model.graph.input:
    if inp.name in name_map:
        # Create a new ValueInfoProto with the new name
        new_inp = helper.make_tensor_value_info(name_map[inp.name],
                                                inp.type.tensor_type.elem_type,
                                                [dim.dim_value for dim in inp.type.tensor_type.shape.dim])
        new_inputs.append(new_inp)
    else:
        new_inputs.append(inp)

# Clear the old inputs and add the new ones
onnx_model.graph.ClearField("input")
onnx_model.graph.input.extend(new_inputs)

# Go through all nodes in the model and replace the old input name with the new one
for node in onnx_model.graph.node:
    for i, input_name in enumerate(node.input):
        if input_name in name_map:
            node.input[i] = name_map[input_name]

# Save the renamed ONNX model
onnx.save(onnx_model, 'model-new.onnx')